In [1]:
from optparse import OptionParser
from sklearn.cluster import KMeans
import numpy as np
import sys
import codecs
import csv
import gc
from copy import deepcopy
from db_class import db
import csv
from datetime import datetime, date

In [2]:
# Check Args ##
# try:
#     Args = sys.argv
#     if(len(Args)!=3):
#         raise EnvironmentError
# except:
#     print("Error !!")
#     print("Plese add 2 arguments {file:Story_header} {file:Story_header_seller}\n")
#     exit()

In [3]:
#### Read file ####
novel_header = list()
data = list()
label = list()
Group_filter = ["26","27","28","29","30","19"]
index = 0
with open("story_header_m04b.csv","r+",encoding="utf8") as test:
    reader = csv.reader(test)
    for row in reader:
        if(not row[2] in Group_filter):
            novel_header.append(row[0:-4])
            data.append(row[-4:])
            label.append(0)
del novel_header[0]
del data[0]
del label[0]
index = len(novel_header)
with open("story_header_sell.csv","r+",encoding="utf8") as test:
    reader = csv.reader(test)
    for row in reader:
        novel_header.append(row[0:-4])
        data.append(row[-4:])
        label.append(1)
del novel_header[index]
del data[index]
del label[index]

In [4]:
#########################################################
################# Bubble sort ###########################
#########################################################
def sort():
    global Result_value
    global novel_header
    global data
    for x in range(len(Result_value)):
        for y in range(len(Result_value)):
            if(Result_value_all[x]>Result_value_all[y]):
                Result_value_all[x], Result_value_all[y] = Result_value_all[y],Result_value_all[x]
                novel_header[x], novel_header[y] = novel_header[y], novel_header[x]
                data[x], data[y] = data[y], data[x]
                label[x], label[y] = label[y], label[x]

In [5]:
#########################################################
################# function ##############################
#########################################################
def process(predict, cls):
    global final
    global final_all
    global Result_value
    global Result_value_all
    
    if not Result_value_all:
        for k in range(cls):
            count = 0
            count_pls = 0
            for loop in range(len(predict)):
                if(predict[loop]==k):
                    count = count + 1
                    if(label[loop]==1):
                        count_pls = count_pls + 1
            final.append(count_pls)
            final_all.append(count)
            buffer = list()
        for k in range(len(predict)):
            val = (float(final[predict[k]])/float(final_all[predict[k]]))*(1-(float(final_all[predict[k]])/len(train_data)))
            Result_value_all.append(val)
            buffer.append(val)
        Result_value.append(buffer)
        final = list()
        final_all = list()
    else:
        for k in range(cls):
            count = 0
            count_pls = 0
            for loop in range(len(predict)):
                if(predict[loop]==k):
                    count = count + 1
                    if(label[loop]==1):
                        count_pls = count_pls + 1
            final.append(count_pls)
            final_all.append(count)
            buffer = list()
        for k in range(len(predict)):
            val = (float(final[predict[k]])/float(final_all[predict[k]]))*(1-(float(final_all[predict[k]])/len(train_data)))
            buffer.append(val)
            if(val>Result_value_all[k]):
                Result_value_all[k] = val
                
        Result_value.append(buffer)
        final = list()
        final_all = list()

In [6]:
################################################
####################  Main  ####################
################################################
final = list()
final_all = list()
Result_value_all = list()
Result_value = list()
train_data = np.asarray(data)
ite = 119

In [7]:
Trans = np.asarray(data).T.tolist()
Max_value = [float(max(Trans[0])), float(max(Trans[1])), float(max(Trans[2])), float(max(Trans[3]))]
Min_value = [float(min(Trans[0])), float(min(Trans[1])), float(min(Trans[2])), float(min(Trans[3]))]
dif_value = [Max_value[0]-Min_value[0], Max_value[1]-Min_value[1], Max_value[2]-Min_value[2], Max_value[3]-Min_value[3]]
datax = deepcopy(data)
for i in range(len(data)):
    datax[i][0] = (float(datax[i][0]) - Min_value[0]) / dif_value[0]
    datax[i][1] = (float(datax[i][1]) - Min_value[1]) / dif_value[1]
    datax[i][2] = (float(datax[i][2]) - Min_value[2]) / dif_value[2]
    datax[i][3] = (float(datax[i][3]) - Min_value[3]) / dif_value[3]
del Trans
del Min_value
del Max_value
del dif_value
gc.collect()

# convert data to array
train_data = np.asarray(datax)

In [8]:
for i in range(2,ite):
    kmeans = KMeans(n_clusters=i, max_iter=500 ,random_state=0).fit(train_data)
    result = kmeans.labels_
    result = result.tolist()
    process(result, i)
sort()
    
# with codecs.open("Result.csv","w","utf-8") as test:
#     for i in range(0,len(Result_value_all)):
#         if(label[i]==0):
#             head = ",".join(novel_header[i])
#             body = ",".join(data[i])
#             buf = str(head)+","+str(body)+","+str(Result_value_all[i])+"\n"
#             test.write(buf)

In [9]:
# with open("Result_K.csv","w") as test:
#     buf = ""
#     for j in range(0,len(Result_value_all)):
#         if(label[i]==1):
#             for i in range(0,ite-4):
#                 buf = buf + str(Result_value[i][j]) + ","
#             buf = buf + str(Result_value[ite-3][j]) + "\n"
#             test.write(buf)
#             buf = ""

In [12]:
database = db("localhost", 3306,"root","1234", "test")
# for i in range(0,len(Result_value_all)):
for i in range(0,len(Result_value_all)):
    if(label[i]==0):
        try:
            datetime_object = datetime.strptime(novel_header[i][7], '%d/%m/%Y, %H:%M:%S')
        except:
            datetime_object = novel_header[i][7]
        buffer = "INSERT INTO result SET \
                id=" + str(novel_header[i][0]) + ","+\
                "main_group=" + str(novel_header[i][1]) + ","+\
                "groupa=" + str(novel_header[i][2]) + ","+\
                "username=\"" + str(novel_header[i][3]) + "\","+\
                "title= %s ,"+\
                "type=" + str(novel_header[i][5]) + ","+\
                "is_end=" + str(novel_header[i][6]) + ","+\
                "updated=\"" + str(datetime_object) +"\","+\
                "is_ban=" + str(novel_header[i][8]) + ","+\
                "is_lock=" + str(novel_header[i][9]) + ","+\
                "published=\"" + str(novel_header[i][10]) + "\","+\
                "isbad=" + str(novel_header[i][11]) + ","+\
                "pack_draft=" + str(novel_header[i][12]) + ","+\
                "pack_selling=" + str(novel_header[i][13]) + ","+\
                "status=" + str(novel_header[i][14]) + ","+\
                "fav=" + str(data[i][0]) + ","+\
                "chapter=" + str(data[i][1]) + ","+\
                "visitor_all=" + str(data[i][2]) + ","+\
                "visitor=" + str(data[i][3]) + ","+\
                "result_value=" + str(Result_value_all[i])+ ","+\
                "label=" + str(0)
        database.CURD_SP(buffer, novel_header[i][4])

Error:  (1062, "Duplicate entry '1616748' for key 'PRIMARY'")
Error:  (1062, "Duplicate entry '1615063' for key 'PRIMARY'")
Error:  (1062, "Duplicate entry '1627376' for key 'PRIMARY'")
Error:  (1062, "Duplicate entry '1616723' for key 'PRIMARY'")
Error:  (1062, "Duplicate entry '1619510' for key 'PRIMARY'")
Error:  (1062, "Duplicate entry '1624401' for key 'PRIMARY'")
Error:  (1062, "Duplicate entry '1630647' for key 'PRIMARY'")
Error:  (1062, "Duplicate entry '1631297' for key 'PRIMARY'")


In [11]:
del database